In [ ]:
# !pip install pandas 
# !pip install -U scikit-learn
# !pip install scipy


### If there is any issue with tensorflow installation, please follow this:
https://gist.github.com/automata/09fd4e1e64118dda4a79559898dd5022

In [1]:
import utils,keras_model,os,random
import tensorflow as tf
import create_dataset
import numpy as np

from keras.optimizers import Adam
from keras.models import load_model
from keras.utils import multi_gpu_model
from sklearn.metrics import r2_score
import seaborn as sns
import pandas as pd

from nilearn.image import resample_img,concat_imgs,load_img,resample_to_img

# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_random_seed(seed_value)

# tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# from tensorflow.keras import backend as K
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/myclone/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anac

In [2]:
#######################################################################################################################
data_path = '/home/ubuntu/hacking/data/stephan-data-ni-files'
input_shape = (91,109,91,1)
result_path = '/home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb'
source_img_name = None
batchSize = 64
ngpus = 2
nEpochs = 1000
decayRate = 0.01
lr = 0.001
pat = 50
num_runs = 1
iterations = 20
use_transfer_learning = False
use_dynamic_LR=False
optimizer='Adam'

if os.path.isdir(result_path):
    print("result directory exists...")
else:
    print("Creating the result directory")
    os.mkdir(result_path)

result directory exists...


In [3]:
######################################################################################################################

dataset = create_dataset.generate_dataset(data_path,test_data_size=0.30)
train_rate,train_temp,train_imgs_list,flat_train_rate,flat_train_rate_zs = utils.get_rate_temp_img(dataset,subjs='train_subjs')
# val_rate,val_temp,val_imgs_list,flat_val_rate,flat_val_rate_zs = utils.get_rate_temp_img(dataset,subjs='val_subjs')
test_rate,test_temp,test_imgs_list,flat_test_rate,flat_test_rate_zs = utils.get_rate_temp_img(dataset,subjs='test_subjs')

df_train = pd.DataFrame()
df_test = pd.DataFrame()
df_val = pd.DataFrame()

df_train['X'] = train_temp
df_train['Y'] = flat_train_rate_zs

# df_val['X'] = val_temp
# df_val['Y'] = flat_val_rate_zs

df_test['X'] = test_temp
df_test['Y'] = flat_test_rate_zs



These are the studies to use ['NSF', 'BMRK3', 'BMRK4', 'IE', 'ILCP', 'EXP', 'SCEBL']
This is the test study BMRK5 


In [7]:
#####################################################################################################################
# train_imgs_list[0:5],val_imgs_list[0:5],test_imgs_list[0:5]
#####################################################################################################################
train_imgs_list[0:5],test_imgs_list[0:5]

(['/home/ubuntu/hacking/data/stephan-data-ni-files/NSF_data/nsf_subj1_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/NSF_data/nsf_subj2_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/NSF_data/nsf_subj3_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/NSF_data/nsf_subj4_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/NSF_data/nsf_subj5_zs.nii'],
 ['/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S1000_OC1474_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S1024_OC1634_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S1031_OC1524_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S1033_OC1490_zs.nii',
  '/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S1035_OC1508_zs.nii'])

In [ ]:
!nvidia-smi

In [9]:
print("Reading training images...")
train_imgs = concat_imgs(train_imgs_list)
train_imgs = np.rollaxis(train_imgs.get_fdata(), 3, 0)[...,None]

# print("Reading validation images...")
# val_imgs = concat_imgs(val_imgs_list)
# val_imgs = np.rollaxis(val_imgs.get_fdata(), 3, 0)[...,None]
val_imgs  = None

print("Reading testing images...")
test_imgs = concat_imgs(test_imgs_list)
test_imgs = np.rollaxis(test_imgs.get_fdata(), 3, 0)[...,None]

# print(train_imgs.shape,val_imgs.shape,test_imgs.shape)
input_shape = train_imgs.shape[1:]

pre_model = None #'/home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-5.h5'


Reading training images...
Reading testing images...


In [ ]:
#####################################################################################################################


print("######################## Loading the model #####################")
if pre_model == None:
    model = keras_model.resnet_with_batchnorm(input_shape)
    model.summary()
else:
    print("Loading model %s"%pre_model)
    model = load_model(pre_model)
    model = model.layers[-2] #specifically for tf1.12. The model is saved with lambda layers. 
    
if ngpus > 1:
    parallel_model = multi_gpu_model(model,gpus=ngpus)
else:
    parallel_model = model
    print("################Creating the model###################")
    
opt = Adam(lr, beta_1=0.9, beta_2=0.999,decay=decayRate)
parallel_model.compile(loss='mean_absolute_error',optimizer=opt)

print("################################ Start mediation analysis ##################################")
output_df = utils.create_empty_df(num_runs,iterations)
for runs in range(num_runs):
    params_df,z_final_train,z_final_val,parallel_model_final = utils.simulate_mediation(df_train,
                                df_val,df_test,train_imgs,val_imgs,test_imgs,output_df,
                                parallel_model,runs,batchSize,nEpochs,iterations,pat,
                                output_path=result_path,use_model=pre_model)
      
    
############################################################################################################   
############################################################################################################


# strategy = tf.contrib.distribute.MirroredStrategy()
# parallel_model = keras_model.resnet_with_batchnorm(input_shape)
# print('###################### Number of devices: {}######################'.format(strategy.worker_devices))
# with strategy.scope():
#     print("################Creating the model###################")
# #     parallel_model = keras_model.resnet_with_batchnorm(input_shape)
#     opt = Adam(lr, beta_1=0.9, beta_2=0.999,decay=decayRate)
#     parallel_model.compile(loss='mean_absolute_error',optimizer=opt)

#     print("################################ Start mediation analysis ##################################")
#     output_df = utils.create_empty_df(num_runs,iterations)
#     for runs in range(num_runs):
#         params_df,z_final_train,z_final_val,parallel_model_final = utils.simulate_mediation(df_train,
#                                     df_val,df_test,train_imgs,val_imgs,test_imgs,output_df,
#                                     parallel_model,runs,batchSize,nEpochs,iterations,pat,
#                                     output_path=result_path,use_model=pre_model)


######################## Loading the model #####################
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Allimages (InputLayer)          (None, 91, 109, 91,  0                                            
__________________________________________________________________________________________________
conv3d_16 (Conv3D)              (None, 91, 109, 91,  224         Allimages[0][0]                  
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 91, 109, 91,  32          conv3d_16[0][0]                  
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 91, 109, 91,  0           batch_normalization_11[0][0]     
____________________________________________

################################ Start mediation analysis ##################################
Using intial random model parameters
Starting iteration... 1
Train on 9360 samples, validate on 4012 samples
Epoch 1/1000
9360/9360 [==============================] - 320s 34ms/step - loss: 3.0682 - val_loss: 0.1070

Epoch 00001: val_loss improved from inf to 0.10701, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 2/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 1.8877 - val_loss: 0.1002

Epoch 00002: val_loss improved from 0.10701 to 0.10019, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 3/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 1.7162 - val_loss: 0.0996

Epoch 00003: val_loss improved from 0.10019 to 0.09958, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 4/1000
936

9360/9360 [==============================] - 306s 33ms/step - loss: 0.1700 - val_loss: 0.0943

Epoch 00045: val_loss did not improve from 0.09291
Epoch 46/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1576 - val_loss: 0.0926

Epoch 00046: val_loss improved from 0.09291 to 0.09262, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 47/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1447 - val_loss: 0.0923

Epoch 00047: val_loss improved from 0.09262 to 0.09227, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 48/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1355 - val_loss: 0.0910

Epoch 00048: val_loss improved from 0.09227 to 0.09105, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 49/1000
9360/9360 [==============================] - 306s 3

9360/9360 [==============================] - 306s 33ms/step - loss: 0.0524 - val_loss: 0.0628

Epoch 00111: val_loss improved from 0.06304 to 0.06281, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 112/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0521 - val_loss: 0.0632

Epoch 00112: val_loss did not improve from 0.06281
Epoch 113/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0520 - val_loss: 0.0630

Epoch 00113: val_loss did not improve from 0.06281
Epoch 114/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0515 - val_loss: 0.0622

Epoch 00114: val_loss improved from 0.06281 to 0.06217, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 115/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0513 - val_loss: 0.0620

Epoch 00115: val_loss improved from 0.06217 to 0.06203,

9360/9360 [==============================] - 306s 33ms/step - loss: 0.0443 - val_loss: 0.0587

Epoch 00150: val_loss did not improve from 0.05850
Epoch 151/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0439 - val_loss: 0.0589

Epoch 00151: val_loss did not improve from 0.05850
Epoch 152/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0438 - val_loss: 0.0585

Epoch 00152: val_loss improved from 0.05850 to 0.05846, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5
Epoch 153/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0436 - val_loss: 0.0585

Epoch 00153: val_loss did not improve from 0.05846
Epoch 154/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0434 - val_loss: 0.0587

Epoch 00154: val_loss did not improve from 0.05846
Epoch 155/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.0433 - val_loss: 0.058

...


Epoch 00083: val_loss did not improve from 0.65346
Epoch 84/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1082 - val_loss: 0.6589

Epoch 00084: val_loss did not improve from 0.65346
Epoch 85/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1064 - val_loss: 0.6593

Epoch 00085: val_loss did not improve from 0.65346
Epoch 86/1000
9360/9360 [==============================] - 305s 33ms/step - loss: 0.1067 - val_loss: 0.6594

Epoch 00086: val_loss did not improve from 0.65346
Epoch 87/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1064 - val_loss: 0.6599

Epoch 00087: val_loss did not improve from 0.65346
Epoch 88/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1073 - val_loss: 0.6621

Epoch 00088: val_loss did not improve from 0.65346
Epoch 89/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1069 - val_loss: 0.6582

Epoch 00089: val_loss did not improve f

9360/9360 [==============================] - 305s 33ms/step - loss: 0.1074 - val_loss: 0.6604

Epoch 00038: val_loss did not improve from 0.65937
Epoch 39/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1073 - val_loss: 0.6658

Epoch 00039: val_loss did not improve from 0.65937
Epoch 40/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1073 - val_loss: 0.6653

Epoch 00040: val_loss did not improve from 0.65937
Epoch 41/1000
9360/9360 [==============================] - 305s 33ms/step - loss: 0.1079 - val_loss: 0.6670

Epoch 00041: val_loss did not improve from 0.65937
Epoch 42/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1061 - val_loss: 0.6715

Epoch 00042: val_loss did not improve from 0.65937
Epoch 43/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1074 - val_loss: 0.6716

Epoch 00043: val_loss did not improve from 0.65937
Epoch 44/1000
9360/9360 [=============================

9360/9360 [==============================] - 306s 33ms/step - loss: 0.1081 - val_loss: 0.6700

Epoch 00012: val_loss did not improve from 0.66220
Epoch 13/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1073 - val_loss: 0.6706

Epoch 00013: val_loss did not improve from 0.66220
Epoch 14/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1073 - val_loss: 0.6695

Epoch 00014: val_loss did not improve from 0.66220
Epoch 15/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1050 - val_loss: 0.6695

Epoch 00015: val_loss did not improve from 0.66220
Epoch 16/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1062 - val_loss: 0.6684

Epoch 00016: val_loss did not improve from 0.66220
Epoch 17/1000
9360/9360 [==============================] - 305s 33ms/step - loss: 0.1066 - val_loss: 0.6717

Epoch 00017: val_loss did not improve from 0.66220
Epoch 18/1000
9360/9360 [=============================


Epoch 00001: val_loss improved from inf to 0.66749, saving model to /home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-14.h5
Epoch 2/1000
9360/9360 [==============================] - 305s 33ms/step - loss: 0.1069 - val_loss: 0.6743

Epoch 00002: val_loss did not improve from 0.66749
Epoch 3/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1063 - val_loss: 0.6704

Epoch 00003: val_loss did not improve from 0.66749
Epoch 4/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1065 - val_loss: 0.6689

Epoch 00004: val_loss did not improve from 0.66749
Epoch 5/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1084 - val_loss: 0.6768

Epoch 00005: val_loss did not improve from 0.66749
Epoch 6/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1092 - val_loss: 0.6723

Epoch 00006: val_loss did not improve from 0.66749
Epoch 7/1000
9360/9360 [=============================

9360/9360 [==============================] - 306s 33ms/step - loss: 0.1053 - val_loss: 0.6745

Epoch 00051: val_loss did not improve from 0.66693
Epoch 52/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1057 - val_loss: 0.6758

Epoch 00052: val_loss did not improve from 0.66693
Epoch 53/1000
9360/9360 [==============================] - 307s 33ms/step - loss: 0.1064 - val_loss: 0.6693

Epoch 00053: val_loss did not improve from 0.66693
Epoch 54/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1069 - val_loss: 0.6731

Epoch 00054: val_loss did not improve from 0.66693
Epoch 55/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1062 - val_loss: 0.6702

Epoch 00055: val_loss did not improve from 0.66693
Epoch 56/1000
9360/9360 [==============================] - 306s 33ms/step - loss: 0.1056 - val_loss: 0.6724

Epoch 00056: val_loss did not improve from 0.66693
Epoch 57/1000
9360/9360 [=============================

In [ ]:
pre_model = None#'/home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-5.h5'
print("######################## Loading the model #####################")
if pre_model == None:
    model = keras_model.resnet_with_batchnorm(input_shape)
    model.summary()
else:
    print("Loading model %s"%pre_model)
    model = load_model(pre_model)
    model = model.layers[-2] #specifically for tf1.12. The model is saved with lambda layers. 
    
if ngpus > 1:
    parallel_model = multi_gpu_model(model,gpus=ngpus)
else:
    parallel_model = model
    print("################Creating the model###################")
    
opt = Adam(lr, beta_1=0.9, beta_2=0.999,decay=decayRate)
parallel_model.compile(loss='mean_absolute_error',optimizer=opt)

print("################################ Start mediation analysis ##################################")
output_df = utils.create_empty_df(num_runs,iterations)
for runs in range(num_runs):
    params_df,z_final_train,z_final_val,parallel_model_final = utils.simulate_mediation(df_train,
                                df_val,df_test,train_imgs,val_imgs,test_imgs,output_df,
                                parallel_model,runs,batchSize,nEpochs,iterations,pat,
                                output_path=result_path,use_model=pre_model)
      
    

### Concatenate test images path

In [ ]:
from pathlib import Path
test_img_names = []
for i in test_imgs_list:
    test_img_names.append([Path(i).name]*(load_img(i).shape[-1]))
test_img_names = np.concatenate(test_img_names)

In [ ]:
test_img_names.shape

### Create an empty dataframe

In [ ]:
df_test_params=pd.DataFrame(columns = ['alpha0','beta0','alpha','beta','gamma'])
df_test_params.loc[0,'alpha0']=0
df_test_params.loc[0,'beta0']=0
df_test_params.loc[0,'alpha']=0
df_test_params.loc[0,'beta']=0
df_test_params.loc[0,'gamma']=0

In [ ]:
for i in range(1,20):
    model_path = os.path.join(result_path,'model-iter-'+str(i)+'.h5')
    print(model_path)
    model_to_use = load_model(model_path)
    alph0_test,alph_test,beta0_test,bet_test,gam_test,z_test = utils.predict_mediation(model_to_use,test_imgs,
                                                                                       df_test)
    df_test['Z'] = z_test
    df_test['image-name'] = test_img_names
    df_test.to_excel(os.path.join(result_path,'df_test_using_model_'+str(i)+'.xlsx'))
    
    df_test_params=pd.DataFrame(columns = ['alpha0','beta0','alpha','beta','gamma'])
    df_test_params.loc[0,'alpha0']=alph0_test
    df_test_params.loc[0,'beta0']=beta0_test
    df_test_params.loc[0,'alpha']=alph_test
    df_test_params.loc[0,'beta']=bet_test
    df_test_params.loc[0,'gamma']=gam_test
#     df_test_params['alpha0'] = alph0_test
#     df_test_params['beta0'] = beta0_test
#     df_test_params['alpha'] = alph_test
#     df_test_params['beta'] = bet_test
#     df_test_params['gamma'] = gam_test
    df_test_params.to_excel(os.path.join(result_path,'params_df_test_model_'+str(i)+'.xlsx'))

In [ ]:
model_path = '/home/ubuntu/hacking/projects/deep-mediation/dec-2020/results/attempt-0/model-iter-3.h5'
model_to_use = load_model(model_path)

In [ ]:
alph0_test,alph_test,beta0_test,bet_test,gam_test,z_test = utils.predict_mediation(model_to_use,test_imgs,df_test)

In [ ]:
df_test['Z'] = z_test
df_test['image-name'] = test_img_names
df_test.to_excel(os.path.join(result_path,'df_test_using_model_3_jan-14.xlsx'))

df_test_params=pd.DataFrame()
df_test_params['alpha0'] = alph0_test
df_test_params['beta0'] = beta0_test
df_test_params['alpha'] = alph_test
df_test_params['beta'] = bet_test
df_test_params['gamma'] = gam_test
df_test_params.to_excel(os.path.join(result_path,'params_df_test_model_3_jan-14.xlsx'))

In [ ]:
model_path = '/home/ubuntu/hacking/projects/deep-mediation/dec-2020/results/OLD/model-iter-3.h5'
print(model_path)
model_to_use = load_model(model_path)
alph0_test,alph_test,beta0_test,bet_test,gam_test,z_test = utils.predict_mediation(model_to_use,test_imgs,
                                                                                   df_test)
df_test['Z'] = z_test
df_test['image-name'] = test_img_names
# df_test.to_excel(os.path.join(result_path,'df_test_using_model_'+str(i)+'.xlsx'))

### Sound data

In [ ]:
data_path = '/home/ubuntu/hacking/data/stephan-data-ni-files/sound_data'
sound_temp_files,sound_ratings_files,sound_images_files = utils.get_sound_files(data_path)

In [ ]:
from nilearn.image import load_img
load_img(sound_images_files[0]).shape

In [ ]:
sound_temp_files,sound_images_files,sound_ratings_files

In [ ]:
temp_mat_file = utils.read_sound_files(sound_temp_files,kind='temp')
rating_mat_file = utils.read_sound_files(sound_ratings_files,kind='rate')
sound_images = utils.read_sound_files(sound_images_files,kind='nii')
sound_images = np.rollaxis(sound_images.get_fdata(), 3, 0)[...,None]

In [ ]:
from pathlib import Path
sound_images_names = []
for i in sound_images_files:
    sound_images_names.append([Path(i).name]*(load_img(i).shape[-1]))
sound_images_names = np.concatenate(sound_images_names)


In [ ]:
df_sound = pd.DataFrame()
df_sound['X'] = temp_mat_file
df_sound['Y'] = rating_mat_file

In [ ]:
import glob
models = glob.glob(os.path.join(result_path,'*.h5'))
models.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

In [ ]:
for i in models:
    model_number = str(Path(i).stem).split('-')[-1]
    print(i,model_number)
    model_to_use = load_model(i)
    alph0_test,alph_test,beta0_test,bet_test,gam_test,z_test = utils.predict_mediation(model_to_use,sound_images,
                                                                                       df_sound)
    df_sound['Z'] = z_test
    df_sound['image-name'] = sound_images_names

    df_sound.to_excel(os.path.join(result_path,'df_sound_using_model_'+model_number+'.xlsx'))